In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%cd /storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES/OLIVES

/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES/OLIVES


In [3]:
import pandas as pd

# Load the CSV file
labels_file = "Biomarker_Clinical_Data_Images.csv"
df = pd.read_csv(labels_file)

# Replace spaces with underscores and remove leading slashes
df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'] = (
    df['Path (Trial/Arm/Folder/Visit/Eye/Image Name)']
    .str.replace(" ", "_")    # Replace spaces with underscores
    .str.lstrip("/")          # Remove leading slash
)

# Save the updated CSV file
updated_labels_file = "Biomarker_Clinical_Data_Images_Updated.csv"
df.to_csv(updated_labels_file, index=False)

print(f"Updated CSV file saved to: {updated_labels_file}")

Updated CSV file saved to: Biomarker_Clinical_Data_Images_Updated.csv


In [3]:
!pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
!export CUDA_LAUNCH_BLOCKING=1

In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, AutoImageProcessor
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tqdm import tqdm

# ============================
# 1. Load and Process Dataset
# ============================

# Load labels file
labels_file = "Biomarker_Clinical_Data_Images_Updated.csv"
df = pd.read_csv(labels_file)

# Ensure biomarkers and clinical labels are numeric
biomarker_columns = df.columns[3:19]
clinical_label_columns = df.columns[19:]

# Convert biomarker and clinical label columns to numeric, handling errors
df[biomarker_columns] = df[biomarker_columns].apply(pd.to_numeric, errors='coerce')
df[clinical_label_columns] = df[clinical_label_columns].apply(pd.to_numeric, errors='coerce')

# Fill any NaN values with a default (e.g., 0 for biomarkers)
df[biomarker_columns] = df[biomarker_columns].fillna(0)
df[clinical_label_columns] = df[clinical_label_columns].fillna(0)

# Normalize biomarkers to [0, 1] range
df[biomarker_columns] = df[biomarker_columns] / df[biomarker_columns].max()

# Validate that labels are in the range [0, 1]
assert df[biomarker_columns].max().max() <= 1, "Biomarkers contain values greater than 1."
assert df[biomarker_columns].min().min() >= 0, "Biomarkers contain values less than 0."

# Split data into train/test based on unique Patient_ID
train_patients, test_patients = train_test_split(df['Patient_ID'].unique(), test_size=0.2, random_state=42)
train_df = df[df['Patient_ID'].isin(train_patients)]
test_df = df[df['Patient_ID'].isin(test_patients)]

# ============================
# 2. Dataset Class
# ============================

class OCTDataset(Dataset):
    def __init__(self, dataframe, is_multimodal=False):
        self.dataframe = dataframe
        self.is_multimodal = is_multimodal
        self.feature_extractor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        # Use the relative path directly since the current directory is the base directory
        relative_path = row['Path (Trial/Arm/Folder/Visit/Eye/Image Name)'].strip()
        image_path = os.path.abspath(relative_path)

        # Check if the file exists
        if not os.path.exists(image_path):
            print(f"Path does not exist: {image_path}")
            return None

        # Load the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Error reading image: {image_path}")
            return None

        # Resize and prepare the image
        image = cv2.resize(image, (224, 224))
        image = np.stack([image] * 3, axis=-1)  # Convert to 3 channels
        image = self.feature_extractor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)

        # Prepare labels
        biomarkers = torch.tensor(row.iloc[3:19].values.astype(np.float32), dtype=torch.float32)
        if self.is_multimodal:
            clinical_labels = torch.tensor(row.iloc[19:].values.astype(np.float32), dtype=torch.float32)
            return image, biomarkers, clinical_labels
        return image, biomarkers


# Initialize datasets
train_dataset = OCTDataset(train_df, is_multimodal=False)
test_dataset = OCTDataset(test_df, is_multimodal=False)

# Remove None entries from dataset
train_dataset = [item for item in train_dataset if item is not None]
test_dataset = [item for item in test_dataset if item is not None]

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ============================
# 3. Vision Transformer Model
# ============================

# Load Vision Transformer with ignore_mismatched_sizes
vit_model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=16,
    ignore_mismatched_sizes=True  # Handle size mismatch for the classifier layer
)

# Replace the classifier for multi-label classification
vit_model.classifier = nn.Sequential(
    nn.Linear(vit_model.config.hidden_size, 16),
    nn.Sigmoid()
)

# Initialize the new classifier's weights
def initialize_weights(module):
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        nn.init.zeros_(module.bias)

vit_model.classifier.apply(initialize_weights)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_model = vit_model.to(device)

print(f"Updated model classifier: {vit_model.classifier}")

# ============================
# 4. Training Loop
# ============================

def train_model(model, loader, optimizer, criterion, epochs=30):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(loader):
            if not batch:  # Handle empty batches
                continue

            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(loader):.4f}")


# Train the model
optimizer = optim.Adam(vit_model.parameters(), lr=1e-4)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for multi-label classification
train_model(vit_model, train_loader, optimizer, criterion)

# ============================
# 5. Evaluation
# ============================

def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in loader:
            if not batch:  # Handle empty batches
                continue

            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            y_true.append(labels.cpu().numpy())
            y_pred.append(outputs.cpu().numpy())
    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)
    return y_true, y_pred

# Evaluate
y_true, y_pred = evaluate_model(vit_model, test_loader)

# Convert predicted probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Ensure `y_true` is binary
y_true_binary = (y_true > 0.5).astype(int)

# Metrics
print("Classification Report:")
print(classification_report(y_true_binary, y_pred_binary))

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([16]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([16, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Updated model classifier: Sequential(
  (0): Linear(in_features=768, out_features=16, bias=True)
  (1): Sigmoid()
)


100%|██████████| 460/460 [00:11<00:00, 41.59it/s]


Epoch 1/30, Loss: 0.2096


100%|██████████| 460/460 [00:10<00:00, 41.84it/s]


...

100%|██████████| 460/460 [00:10<00:00, 41.84it/s]


Epoch 29/30, Loss: 0.0358


100%|██████████| 460/460 [00:11<00:00, 41.79it/s]


Epoch 30/30, Loss: 0.0385
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.26      0.39       159
           1       0.00      0.00      0.00        19
           2       0.04      0.01      0.02        70
           3       0.91      0.83      0.87      1530
           4       0.78      0.60      0.68       684
           5       0.83      0.95      0.89      1278
           6       0.00      0.00      0.00        32
           7       0.66      0.54      0.59       558
           8       0.00      0.00      0.00         0
           9       0.85      0.88      0.87       721
          10       0.90      0.83      0.86       987
          11       0.92      0.41      0.57        58
          12       0.00      0.00      0.00         6
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00        34
          15       0.77      0.76      0.76       980

   micro avg       0.83      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me